In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

import json
import pickle
import logging

import pandas as pd
import numpy as np

from itext2kg import iText2KG
from itext2kg.utils import PubtatorProcessor
from langchain_ollama import ChatOllama, OllamaEmbeddings
from itext2kg.graph_integration import GraphIntegrator

/home/jovyan/miniconda3/envs/kg/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function array> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
NEO4J_URI = "neo4j+s://435eb4f0.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "nqL3At3rptFpfRrBhg5Y1-7k78oHqVaW42WKtqVRnJo"

llm = ChatOllama(
    model="deepseek-r1:32b",
    temperature=0,
)

embeddings = OllamaEmbeddings(
    model="nomic-embed-text:latest",
)

In [ ]:
df1 = pd.read_csv("/home/jovyan/my_code/RAG/AD_pmid_results.csv")
pmid_list = list(df1['pmid'].astype(str))
pmid_list = pmid_list[:10]  
PUTATOR_PATH = "/home/jovyan/my_code/RAG/Data_v2/AD_pubtabor/"
JSON_PATH = "/home/jovyan/my_code/itext2kg/output_kg/AD"
OUTPUT_PATH = JSON_PATH

In [ ]:
pmid_list

['39652311',
 '39890535',
 '39855320',
 '39589570',
 '39604278',
 '39596657',
 '39568323',
 '39657091',
 '39276711',
 '39431763']

In [ ]:
PMID = pmid_list[1]  # 这个 PMID 提取出来全是散的节点，没有的关系，重新跑一下，看看问题在哪里了

In [ ]:
with open(f"{PUTATOR_PATH}{PMID}.txt", "r") as f:
    text = f.readlines()

In [ ]:
text

["39890535|t|The associations between fresh vegetable and fruit consumption and plasma and PET biomarkers in preclinical Alzheimer's disease: A cross-sectional and longitudinal study of Chinese population.\n",
 "39890535|a|BACKGROUND: The identification of the modifiable lifestyle factors including dietary habits in older adults of preclinical Alzheimer's disease (AD) and early effective interventions are of great importance. OBJECTIVES: We studied whether the consumption of fresh vegetables and fruits was different between cognitively unimpaired (CU) and cognitively impaired (CI) population and mainly investigated the associations between vegetable and fruit consumption and PET and plasma AD biomarkers in older CU adults with higher beta-amyloid (Abeta) burden. DESIGN, SETTING, AND PARTICIPANTS: Older adults with the age of 50-85 years were enrolled for a cross-sectional and longitudinal study. The groups depended on whether the participants were CU or CI. Partial participants whose h

In [ ]:
pubtator_file = f"{PUTATOR_PATH}{PMID}.txt"
pubtator_process = PubtatorProcessor(pubtator_file, llm)
semantic_blocks = pubtator_process.block
properties_info = pubtator_process.properties_info
pubtator_info = pubtator_process.pubtator_info

2025-03-03 18:36:54,735 - INFO - Processing PubTator file: /home/jovyan/my_code/RAG/Data_v2/AD_pubtabor/39890535.txt
2025-03-03 18:36:54,737 - WARNING - Skipping malformed entity line: ['']


2025-03-03 18:36:57,344 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-03 18:37:27,183 - INFO - Abstract distilled successfully.
2025-03-03 18:37:27,186 - INFO - Adding Alzheimer's disease to the abstract
2025-03-03 18:37:27,187 - INFO - Adding AD to the abstract
2025-03-03 18:37:27,187 - INFO - Adding cognitively impaired to the abstract
2025-03-03 18:37:27,188 - INFO - Adding CI to the abstract
2025-03-03 18:37:27,189 - INFO - Adding cognitive decline to the abstract
2025-03-03 18:37:27,189 - INFO - Adding [{'gene': 'Abeta'}, {'gene': 'tau'}, {'gene': 'Tau'}] to the abstract
2025-03-03 18:37:27,190 - INFO - Adding [{'chemical': '18F-florbetapir'}, {'chemical': '18F-MK6240'}] to the abstract


In [ ]:
properties_info

{'source': 'PMID39890535'}

In [ ]:
semantic_blocks

['disease - [[\'disease\': "Alzheimer\'s disease"], [\'disease\': "Alzheimer\'s disease"], [\'disease\': \'AD\'], [\'disease\': \'cognitively impaired\'], [\'disease\': \'CI\'], [\'disease\': \'cognitive decline\']]',
 "gene - [['gene': 'Abeta'], ['gene': 'tau'], ['gene': 'Tau']]",
 "chemical - [['chemical': '18F-florbetapir'], ['chemical': '18F-MK6240']]",
 "Title: The associations between fresh vegetable and fruit consumption and plasma and PET biomarkers in preclinical Alzheimer's disease: A cross-sectional and longitudinal study of Chinese population. Abstract: BACKGROUND: The identification of the modifiable lifestyle factors including dietary habits in older adults of preclinical Alzheimer's disease (AD) and early effective interventions are of great importance. OBJECTIVES: We studied whether the consumption of fresh vegetables and fruits was different between cognitively unimpaired (CU) and cognitively impaired (CI) population and mainly investigated the associations between veg

In [ ]:
pubtator_info['abstract'] = {'context':semantic_blocks[-1], 'source':properties_info['source']}

In [ ]:
pubtator_info

{"Alzheimer's disease": {'label': 'disease', 'unique_id': 'MESH:D000544'},
 'AD': {'label': 'disease', 'unique_id': 'MESH:D000544'},
 'cognitively impaired': {'label': 'disease', 'unique_id': 'MESH:D003072'},
 'CI': {'label': 'disease', 'unique_id': 'MESH:D003072'},
 'Abeta': {'label': 'gene', 'unique_id': 'Gene ID:351'},
 '18F-florbetapir': {'label': 'chemical', 'unique_id': 'MESH:C545186'},
 '18F-MK6240': {'label': 'chemical', 'unique_id': '-'},
 'tau': {'label': 'gene', 'unique_id': 'Gene ID:4137'},
 'Tau': {'label': 'gene', 'unique_id': 'Gene ID:4137'},
 'cognitive decline': {'label': 'disease', 'unique_id': 'MESH:D003072'},
 'abstract': {'context': "Title: The associations between fresh vegetable and fruit consumption and plasma and PET biomarkers in preclinical Alzheimer's disease: A cross-sectional and longitudinal study of Chinese population. Abstract: BACKGROUND: The identification of the modifiable lifestyle factors including dietary habits in older adults of preclinical Alzh

In [ ]:
# -----------------Build the graph-----------------
itext2kg = iText2KG(llm_model = llm, embeddings_model = embeddings)
kg = itext2kg.build_graph(
    sections=[semantic_blocks], 
    source=properties_info,  
    entities_info=pubtator_info,
    ent_threshold=0.9, 
    rel_threshold=0.4,
    )

2025-03-03 18:37:27,400 - INFO - Extracting Entities from the Document


2025-03-03 18:37:29,019 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-03 18:38:04,593 - INFO - INFO --- alzheimer's disease add unique_id and label
2025-03-03 18:38:04,594 - INFO - INFO --- ad add unique_id and label
2025-03-03 18:38:04,595 - INFO - INFO --- cognitively impaired add unique_id and label
2025-03-03 18:38:04,595 - INFO - INFO --- ci add unique_id and label
2025-03-03 18:38:04,596 - INFO - INFO --- cognitive decline add unique_id and label
2025-03-03 18:38:04,597 - INFO - INFO --- abeta add unique_id and label
2025-03-03 18:38:04,597 - INFO - INFO --- tau add unique_id and label
2025-03-03 18:38:04,598 - INFO - INFO --- tau add unique_id and label
2025-03-03 18:38:04,598 - INFO - INFO --- 18f-florbetapir add unique_id and label
2025-03-03 18:38:04,599 - INFO - INFO --- 18f-mk6240 add unique_id and label
2025-03-03 18:38:04,599 - INFO - {'entities': [{'label': 'disease', 'name': "Alzheimer's disease", 'properties': {'unique_id': 'MESH

In [ ]:
GraphIntegrator(uri=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD).visualize_graph(knowledge_graph=kg)

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Variable `MESH` not defined (line 1, column 14087 (offset: 14086))
"CREATE (n:chemical {name: "18f florbetapir"}) SET n.embeddings = [0.014416614300000002, -0.021422587, -0.155926216, 0.013058233199999998, 0.014543733900000001, 0.0120364164, -0.022469499400000002, -0.033633148599999996, 0.0017816480840000001, -0.053918202, -0.011405891799999998, 0.024207019156000004, 0.0573646938, -0.0261367178, 0.0218350978, 0.0272393084, 0.0020948282360000003, -0.0649563068, 0.0156892736, 0.045699137599999995, -0.039309175, -0.04234768, -0.047459344135999997, 0.008355326, 0.0838088136, 0.0252410588, -0.020784920420000002, 0.0382029854, -0.023779700400000003, -0.011934146399999998, -0.00866095738, 0.037574536, -0.0341327888, -0.005087642999999999, 0.0238024432, -0.0651360476, 0.0148338726, 0.0434884276, 0.050224732800000005, 0.02043434776, 0.0256551468, 0.0070232724200000015, 0.0148134882, -0.0316451208, -0.0025634172199999997,